# Homework 4

In [1]:
from importlib import reload

import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns

import unyt

In [2]:
sns.set_theme()
sns.set_style('whitegrid')
sns.set_context('notebook')

# Useful constants

In [4]:
stefan = 5.670374419184429e-8 * unyt.J / unyt.m**2 / unyt.s / unyt.K**4

# Problem 1 -- Leaf

## Our leaf energy budget is:

## $R_n = Q_a - 2 \epsilon_l \sigma T_l^4$

## Code

In [5]:
Qa = 1200 * unyt.W / unyt.m**2
epsilon = 0.98
T_l = (30*unyt.Celsius).to('K')

In [6]:
R_n = Qa - 2 * epsilon * stefan * T_l**4
R_n

unyt_quantity(261.36207351, 'W/m**2')

----

# Problem 2 -- Boundary layer conductance

## We were presented an approximate formulation of the boudnary layer conductance $g_{bw}$

## $g_{bw} = 0.233 \left ( \frac{u}{\ell} \right )^{1/2} $

## Code

In [16]:
def calc_gbw(l, u):
    return 0.233 * (u/l)**(1/2)

## (a) Small + calm

In [17]:
l_a = 3 * unyt.cm
u_a = 0.1 * unyt.m/unyt.s

In [18]:
gbw_a = calc_gbw(l_a, u_a)
gbw_a

unyt_quantity(0.42539785, '1/sqrt(s)')

## (b) Small + strong

In [19]:
l_b = 3 * unyt.cm
u_b = 5 * unyt.m/unyt.s

In [20]:
gbw_b = calc_gbw(l_b, u_b)
gbw_b

unyt_quantity(3.00801707, '1/sqrt(s)')

## (c) Large + calm

In [21]:
l_c = 10 * unyt.cm
u_c = 0.1 * unyt.m/unyt.s

In [22]:
gbw_c = calc_gbw(l_c, u_c)
gbw_c

unyt_quantity(0.233, '1/sqrt(s)')

## (d) Large + strong

In [23]:
l_d = 10 * unyt.cm
u_d = 5 * unyt.m/unyt.s

In [24]:
gbw_d = calc_gbw(l_d, u_d)
gbw_d

unyt_quantity(1.6475588, '1/sqrt(s)')

## We see the largest conductance from high wind speeds and small leafs, which follows from our equation ($g_{bw} \propto \sqrt{u/l}$)

----

# Problem 3 -- Penman-Monteith revisited

## In lecture we derived expressions for LE, SH for a leaf:

## $LE = \frac{2 C_p [e^*(T_a) - e_a] g_{bw} + \Delta R_n}{2 \gamma (1+g_{bw}/g_{sw}) + \Delta}$

## $SH = 2 C_p [T_l - T_a] g_{bh}$

## And we have energy balance:

## $R_n = SH + LE$

## We will calculate $LE$, use energy balance to estimate $SH$ and then invert our $SH$ equation for $T_l - T_a$:

## $T_l-T_a = \frac{SH}{2 C_p g_{bh}}$

## We are provided:

In [30]:
g_bw = 2.03 * unyt.mol / unyt.m**2 / unyt.s
estar = 3167 * unyt.Pa # Ta = 25C
Delta = 189 * unyt.Pa / unyt.K  # Ta = 25C 
C_p = 29.2 * unyt.J / unyt.mol / unyt.K  # I think there was a typo, although 29.2 is an odd number too..
gamma = 66.5 * unyt.Pa / unyt.K
RH = 0.75

## Code our equations, recognizing:

## $e_a = RH  e^*$

In [39]:
def calc_LE(g_sw, R_n):
    fterm = 2*C_p*(estar - estar*RH) * g_bw 
    sterm = Delta * R_n
    #import pdb; pdb.set_trace()
    numer = fterm + sterm
    denom = 2 * gamma * (1+g_bw/g_sw) + Delta
    return numer/denom

In [47]:
def calc_TlTs(SH, g_sw):
    return SH/(2 * C_p * g_bw)

## Now we vary $g_{sw}$ and $R_n$

## (a) 

In [37]:
g_sw_a = 0.1 * unyt.mol / unyt.m**2 / unyt.s
R_n_a = 500 * unyt.W / unyt.m**2

In [41]:
LE_a = calc_LE(g_sw_a, R_n_a)
LE_a.to('W/m**2')

unyt_quantity(62.33281909, 'W/m**2')

In [43]:
SH_a = R_n_a - LE_a
SH_a

unyt_quantity(437.66718091, 'W/m**2')

In [52]:
TlTs_a = calc_TlTs(SH_a, g_sw_a)
TlTs_a.to('K')

unyt_quantity(3.69177391, 'K')

## (b)

In [57]:
g_sw_b = 0.1 * unyt.mol / unyt.m**2 / unyt.s
R_n_b = 1000 * unyt.W / unyt.m**2
#
LE_b = calc_LE(g_sw_b, R_n_b)
SH_b = R_n_b - LE_b
TlTs_b = calc_TlTs(SH_b, g_sw_b)
#
print(f"LE={LE_b:0.2f}, SH={SH_b:0.2f}, TlTs={TlTs_b:0.2f}")

LE=93.60 J/(m**2*s), SH=906.40 W/m**2, TlTs=7.65 K*W*s/J


## (c)

In [58]:
g_sw_c = 0.4 * unyt.mol / unyt.m**2 / unyt.s
R_n_c = 500 * unyt.W / unyt.m**2
#
LE_c = calc_LE(g_sw_c, R_n_c)
SH_c = R_n_c - LE_c
TlTs_c = calc_TlTs(SH_c, g_sw_c)
#
print(f"LE={LE_c:0.2f}, SH={SH_c:0.2f}, TlTs={TlTs_c:0.2f}")

LE=188.94 J/(m**2*s), SH=311.06 W/m**2, TlTs=2.62 K*W*s/J


## (d)

In [60]:
g_sw_d = 0.4 * unyt.mol / unyt.m**2 / unyt.s
R_n_d = 1000 * unyt.W / unyt.m**2
#
LE_d = calc_LE(g_sw_d, R_n_d)
SH_d = R_n_d - LE_d
TlTs_d = calc_TlTs(SH_d, g_sw_d)
#
print(f"LE={LE_d:0.2f}, SH={SH_d:0.2f}, TlTs={TlTs_d:0.2f}")

LE=283.72 J/(m**2*s), SH=716.28 W/m**2, TlTs=6.04 K*W*s/J


## We find $LE$ is increasingly higher from a-d

## As regards transpiration and leaf temperature, because $T_l-T_s \propto SH \propto R_n-LE$, lower $LE$ implies higher leaf temperatures.

## And, higher transpiration leads to lower leaf temperature.

## Varying $R_n$ has a direct effect, as $T_l-T_s \propto R_n - LE$, which we see comparing (a) to (b) and (c) to (d) [i.e. fixed $g_{sw}$].

## Net percentage:

In [64]:
p_LE_a = (LE_a/R_n_a).value
p_LE_b = (LE_b/R_n_b).value
p_LE_c = (LE_c/R_n_c).value
p_LE_d = (LE_d/R_n_d).value

In [65]:
print(f"(a) p_LE={p_LE_a:0.2f}")
print(f"(b) p_LE={p_LE_b:0.2f}")
print(f"(c) p_LE={p_LE_c:0.2f}")
print(f"(d) p_LE={p_LE_d:0.2f}")

(a) p_LE=0.12
(b) p_LE=0.09
(c) p_LE=0.38
(d) p_LE=0.28


## For all of these conditions, $SH$ domiantes (its percentage exceeds 50%, and even hits 90%).

## We see an increase percentage for LE (and reduced SH) as we increased $g_{sw}$, which follows given $LE \propto g_{sw}$ for large $g_b$.

## On the other hand, increasing $R_n$ decreases that percentage, i.e. a greater fraction of the heat is released via SH.